1.Отобразите все записи из таблицы company по компаниям, которые закрылись.
```
SELECT *
FROM company
WHERE status = 'closed'
```




2.Отобразите количество привлечённых средств для новостных компаний США. Используйте данные из таблицы company. Отсортируйте таблицу по убыванию значений в поле funding_total.
```
SELECT funding_total
FROM company
WHERE country_code = 'USA'
       AND category_code = 'news'
ORDER BY funding_total DESC
```



3.Найдите общую сумму сделок по покупке одних компаний другими в долларах. Отберите сделки, которые осуществлялись только за наличные с 2011 по 2013 год включительно.
```
SELECT SUM(price_amount)
FROM acquisition
WHERE term_code = 'cash'
  AND EXTRACT(YEAR
              FROM CAST(acquired_at as date)) BETWEEN 2011 AND 2013
```

4.Отобразите имя, фамилию и названия аккаунтов людей в поле network_username, у которых названия аккаунтов начинаются на 'Silver'.
```
SELECT first_name,
       last_name,
       network_username
FROM people
WHERE network_username LIKE 'Silver%'
```

5.Выведите на экран всю информацию о людях, у которых названия аккаунтов в поле network_username содержат подстроку 'money', а фамилия начинается на 'K'.
```
SELECT *
FROM people
WHERE network_username LIKE '%money%'
  AND last_name LIKE 'K%'
```

6.Для каждой страны отобразите общую сумму привлечённых инвестиций, которые получили компании, зарегистрированные в этой стране. Страну, в которой зарегистрирована компания, можно определить по коду страны. Отсортируйте данные по убыванию суммы.
```
SELECT SUM(funding_total),
       country_code
FROM company
GROUP BY country_code
ORDER BY SUM(funding_total) DESC
```

7.Составьте таблицу, в которую войдёт дата проведения раунда, а также минимальное и максимальное значения суммы инвестиций, привлечённых в эту дату.
Оставьте в итоговой таблице только те записи, в которых минимальное значение суммы инвестиций не равно нулю и не равно максимальному значению.
```
SELECT funded_at,
       MIN(raised_amount),
       MAX(raised_amount)
FROM funding_round
GROUP BY funded_at
HAVING MIN(raised_amount) != 0
AND MIN(raised_amount) != MAX(raised_amount)
```

8.Создайте поле с категориями:
Для фондов, которые инвестируют в 100 и более компаний, назначьте категорию high_activity.
Для фондов, которые инвестируют в 20 и более компаний до 100, назначьте категорию middle_activity.
Если количество инвестируемых компаний фонда не достигает 20, назначьте категорию low_activity.
Отобразите все поля таблицы fund и новое поле с категориями.
```
SELECT *,
       CASE
           WHEN invested_companies >= 100 THEN 'high_activity'
           WHEN invested_companies BETWEEN 20 AND 100 THEN 'middle_activity'
           ELSE 'low_activity'
       END
FROM fund
```


9.Для каждой из категорий, назначенных в предыдущем задании, посчитайте округлённое до ближайшего целого числа среднее количество инвестиционных раундов, в которых фонд принимал участие. Выведите на экран категории и среднее число инвестиционных раундов. Отсортируйте таблицу по возрастанию среднего.
```
WITH
activity_t AS (SELECT *,
       CASE
           WHEN invested_companies>=100 THEN 'high_activity'
           WHEN invested_companies>=20 THEN 'middle_activity'
           ELSE 'low_activity'
       END AS activity
FROM fund)

SELECT activity,
       ROUND(AVG(investment_rounds))
FROM activity_t
GROUP BY activity
ORDER BY ROUND(AVG(investment_rounds));
```

10.Проанализируйте, в каких странах находятся фонды, которые чаще всего инвестируют в стартапы.
Для каждой страны посчитайте минимальное, максимальное и среднее число компаний, в которые инвестировали фонды этой страны, основанные с 2010 по 2012 год включительно. Исключите страны с фондами, у которых минимальное число компаний, получивших инвестиции, равно нулю.
Выгрузите десять самых активных стран-инвесторов: отсортируйте таблицу по среднему количеству компаний от большего к меньшему. Затем добавьте сортировку по коду страны в лексикографическом порядке.
```
SELECT country_code,
       MIN(invested_companies),
       MAX(invested_companies),
       AVG(invested_companies) AS avg
FROM fund
WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) BETWEEN 2010 AND 2012
GROUP BY country_code
HAVING MIN(invested_companies) != 0
ORDER BY avg DESC, country_code
LIMIT 10;
```

11.Отобразите имя и фамилию всех сотрудников стартапов. Добавьте поле с названием учебного заведения, которое окончил сотрудник, если эта информация известна.
```
SELECT first_name,
       last_name,
       instituition
FROM people AS p
LEFT JOIN education AS ed ON ed.person_id = p.id
```

12.Для каждой компании найдите количество учебных заведений, которые окончили её сотрудники. Выведите название компании и число уникальных названий учебных заведений. Составьте топ-5 компаний по количеству университетов.
```
WITH inst_count AS
  (SELECT COUNT(DISTINCT instituition),
          company_id
   FROM education AS e
   JOIN people AS p ON p.id = e.person_id
   GROUP BY company_id)
SELECT c.name,
       inst_count.count
FROM company AS c
JOIN inst_count ON inst_count.company_id = c.id
ORDER BY inst_count.count DESC
LIMIT 5;
```

13.Составьте список с уникальными названиями закрытых компаний, для которых первый раунд финансирования оказался последним.
```
WITH first_last AS
   (SELECT company_id
    FROM funding_round
    WHERE is_first_round = 1
      AND is_last_round = 1)
SELECT DISTINCT name
FROM company AS c
RIGHT JOIN first_last ON first_last.company_id = c.id
WHERE status = 'closed'
```

14.Составьте список уникальных номеров сотрудников, которые работают в компаниях, отобранных в предыдущем задании.
```
WITH first_last AS
   (SELECT company_id
    FROM funding_round
    WHERE is_first_round = 1
      AND is_last_round = 1),
     numbers AS
    (SELECT id,
            company_id
    FROM people)
SELECT DISTINCT numbers.id
FROM company AS c
RIGHT JOIN first_last ON first_last.company_id = c.id
RIGHT JOIN numbers ON numbers.company_id = c.id
WHERE status = 'closed';
```

15.Составьте таблицу, куда войдут уникальные пары с номерами сотрудников из предыдущей задачи и учебным заведением, которое окончил сотрудник.
```
WITH first_last AS
   (SELECT company_id
    FROM funding_round
    WHERE is_first_round = 1
      AND is_last_round = 1),
     numbers AS
    (SELECT id,
            company_id
    FROM people),
     inst AS
     (SELECT person_id,
             instituition
     FROM education)
SELECT DISTINCT numbers.id,
                instituition
FROM company AS c
RIGHT JOIN first_last ON first_last.company_id = c.id
RIGHT JOIN numbers ON numbers.company_id = c.id
RIGHT JOIN inst ON inst.person_id = numbers.id
WHERE status = 'closed';
```

16.Посчитайте количество учебных заведений для каждого сотрудника из предыдущего задания. При подсчёте учитывайте, что некоторые сотрудники могли окончить одно и то же заведение дважды.
```
WITH first_last AS
   (SELECT DISTINCT company_id
    FROM funding_round
    WHERE is_first_round = 1
      AND is_last_round = 1),
     numbers AS
    (SELECT id,
            company_id
    FROM people),
     inst AS
    (SELECT person_id,
            instituition
    FROM education)
SELECT numbers.id,
       COUNT(instituition)
FROM company AS c
RIGHT JOIN first_last ON first_last.company_id = c.id
RIGHT JOIN numbers ON numbers.company_id = c.id
RIGHT JOIN inst ON inst.person_id = numbers.id
WHERE status = 'closed'
GROUP BY numbers.id;
```

17.Дополните предыдущий запрос и выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники разных компаний. Нужно вывести только одну запись, группировка здесь не понадобится.
```
WITH first_last AS
   (SELECT DISTINCT company_id
    FROM funding_round
    WHERE is_first_round = 1
      AND is_last_round = 1),
     numbers AS
    (SELECT id,
            company_id
    FROM people),
     inst AS
    (SELECT person_id,
            instituition
    FROM education)
SELECT AVG(inst_count)
FROM
  (SELECT numbers.id,
       COUNT(instituition) AS inst_count
   FROM company AS c
   RIGHT JOIN first_last ON first_last.company_id = c.id
   RIGHT JOIN numbers ON numbers.company_id = c.id
   RIGHT JOIN inst ON inst.person_id = numbers.id
   WHERE status = 'closed'
   GROUP BY numbers.id) AS inst_sum;
```

18.Напишите похожий запрос: выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники Socialnet.
```
WITH first_last AS
   (SELECT DISTINCT company_id
    FROM funding_round),
     numbers AS
    (SELECT id,
            company_id
    FROM people),
     inst AS
    (SELECT person_id,
            instituition
    FROM education)
SELECT AVG(inst_count)
FROM
  (SELECT numbers.id,
       COUNT(instituition) AS inst_count
   FROM company AS c
   RIGHT JOIN first_last ON first_last.company_id = c.id
   RIGHT JOIN numbers ON numbers.company_id = c.id
   RIGHT JOIN inst ON inst.person_id = numbers.id
   WHERE name = 'Socialnet'
   GROUP BY numbers.id) AS inst_sum;
```

19.Составьте таблицу из полей:
name_of_fund — название фонда;
name_of_company — название компании;
amount — сумма инвестиций, которую привлекла компания в раунде.
В таблицу войдут данные о компаниях, в истории которых было больше шести важных этапов, а раунды финансирования проходили с 2012 по 2013 год включительно.
```
WITH fn AS
   (SELECT id,
           name
    FROM fund),
     s AS  
   (SELECT id,
           raised_amount AS amount,
           funded_at
    FROM funding_round),
     c AS
    (SELECT id,
            name AS name_of_company,
            milestones
    FROM company)
SELECT name AS name_of_fund,
       name_of_company,
       amount
FROM investment AS f
LEFT JOIN fn ON f.fund_id = fn.id
LEFT JOIN s ON f.funding_round_id = s.id
LEFT JOIN c ON f.company_id = c.id
WHERE c.milestones > 6
  AND EXTRACT(YEAR FROM funded_at) BETWEEN 2012 AND 2013;
```

20.Выгрузите таблицу, в которой будут такие поля:
название компании-покупателя;
сумма сделки;
название компании, которую купили;
сумма инвестиций, вложенных в купленную компанию;
доля, которая отображает, во сколько раз сумма покупки превысила сумму вложенных в компанию инвестиций, округлённая до ближайшего целого числа.
Не учитывайте те сделки, в которых сумма покупки равна нулю. Если сумма инвестиций в компанию равна нулю, исключите такую компанию из таблицы.
Отсортируйте таблицу по сумме сделки от большей к меньшей, а затем по названию купленной компании в лексикографическом порядке. Ограничьте таблицу первыми десятью записями.
```
WITH c AS
   (SELECT name,
           funding_total,
           id
    FROM company AS c),
     c_2 AS
   (SELECT name,
           id
    FROM company AS c)
SELECT c_2.name AS acquiring_name,
       price_amount,
       c.name AS acquired_name,
       c.funding_total,
       ROUND(price_amount / c.funding_total) AS share
FROM acquisition AS a
LEFT JOIN c ON c.id = a.acquired_company_id
LEFT JOIN c_2 ON c_2.id = a.acquiring_company_id
WHERE price_amount > 0
  AND funding_total > 0
ORDER BY price_amount DESC,
         c.name
LIMIT 10;
```

21.Выгрузите таблицу, в которую войдут названия компаний из категории social, получившие финансирование с 2010 по 2013 год включительно. Проверьте, что сумма инвестиций не равна нулю. Выведите также номер месяца, в котором проходил раунд финансирования.
```
WITH fr AS
   (SELECT EXTRACT(MONTH FROM funded_at) AS month,
           company_id
   FROM funding_round
   WHERE EXTRACT(YEAR FROM funded_at) BETWEEN 2010 AND 2013
     AND raised_amount > 0)
SELECT name,
       fr.month
FROM company AS c
INNER JOIN fr ON fr.company_id = c.id
WHERE category_code = 'social';
```

22.Отберите данные по месяцам с 2010 по 2013 год, когда проходили инвестиционные раунды. Сгруппируйте данные по номеру месяца и получите таблицу, в которой будут поля:
номер месяца, в котором проходили раунды;
количество уникальных названий фондов из США, которые инвестировали в этом месяце;
количество компаний, купленных за этот месяц;
общая сумма сделок по покупкам в этом месяце.
```
WITH fn AS
   (SELECT id,
    country_code
    FROM fund),
     fr AS
    (SELECT id,
            funded_at
    FROM funding_round),
     a AS
    (SELECT acquired_company_id,
            price_amount,
            acquired_at
     FROM acquisition),
     i AS
    (SELECT funding_round_id,
             company_id,
             fund_id
     FROM investment),
inv AS
    (SELECT EXTRACT(MONTH FROM fr.funded_at) AS fund_month,
            COUNT(DISTINCT fn.id) AS funds
    FROM i
    JOIN fr ON fr.id = i.funding_round_id
    JOIN fn ON fn.id = i.fund_id
    WHERE fn.country_code = 'USA'
      AND EXTRACT(YEAR FROM fr.funded_at) BETWEEN 2010 AND 2013
    GROUP BY fund_month),
acq AS
    (SELECT EXTRACT(MONTH FROM acquired_at) AS acq_month,
            COUNT(acquired_company_id) AS acquired_companies,
            SUM(price_amount)
    FROM a
    WHERE EXTRACT(YEAR FROM acquired_at) BETWEEN 2010 AND 2013
    GROUP BY acq_month)

SELECT fund_month,
       funds,
       acquired_companies,
       sum
FROM inv
JOIN  acq ON inv.fund_month = acq.acq_month;




/*
x AS (SELECT fr.month,
       COUNT(price_amount) AS acquired_companies,
       SUM(price_amount)
FROM a
JOIN i ON i.company_id = a.acquired_company_id
JOIN fr ON fr.company_id = i.funding_round_id
WHERE EXTRACT(YEAR FROM funded_at) BETWEEN 2010 AND 2013
GROUP BY fr.month)
*/

/*
SELECT *
FROM x
JOIN xx ON xx.month = x.month
*/

/*
SELECT fr.month,
       fn.name AS funds,
       COUNT(price_amount) AS acquired_companies,
       SUM(price_amount)
FROM company AS c
JOIN investment AS i ON i.company_id = c.id
JOIN fn ON fn.id = i.company_id
JOIN fr ON fr.company_id = c.id
JOIN a ON acquired_company_id = c.id
WHERE EXTRACT(YEAR FROM fr.funded_at) BETWEEN 2010 AND 2013
GROUP BY fr.month, fn.name;
*/
```

23.Составьте сводную таблицу и выведите среднюю сумму инвестиций для стран, в которых есть стартапы, зарегистрированные в 2011, 2012 и 2013 годах. Данные за каждый год должны быть в отдельном поле. Отсортируйте таблицу по среднему значению инвестиций за 2011 год от большего к меньшему.
```
WITH inv_2011 AS
   (SELECT country_code,
           AVG(funding_total) AS funding_total_2011
    FROM company
    WHERE EXTRACT(YEAR FROM founded_at) = 2011
    GROUP BY country_code),
     inv_2012 AS
   (SELECT country_code,
           AVG(funding_total) AS funding_total_2012
    FROM company
    WHERE EXTRACT(YEAR FROM founded_at) = 2012
    GROUP BY country_code),
     inv_2013 AS
   (SELECT country_code,
           AVG(funding_total) AS funding_total_2013
    FROM company
    WHERE EXTRACT(YEAR FROM founded_at) = 2013
    GROUP BY country_code)
SELECT inv_2011.country_code,
       funding_total_2011,
       funding_total_2012,
       funding_total_2013
FROM inv_2011
JOIN inv_2012 ON inv_2012.country_code = inv_2011.country_code
JOIN inv_2013 ON inv_2013.country_code = inv_2011.country_code
ORDER BY funding_total_2011 DESC;
```

Также были выполнены задания дополнительного курса "Дополнительная практика — Python и SQL":
- SQL. База данных Northwind;
- SQL. База данных AdventureWorks.
